# Week 5: Encapsulation & Data Hiding
## High-Low Card Game Analysis

**INST326: Object-Oriented Programming for Information Science**

This notebook demonstrates encapsulation principles through a complete card game implementation. We'll explore:

- **Private attributes** with name mangling (`__attribute`)
- **Read-only properties** using the `@property` decorator
- **Controlled interfaces** that protect data integrity
- **Separation of concerns** between domain logic and presentation

---

## Overview: Three Layers of Encapsulation

This program demonstrates encapsulation at three distinct levels:

1. **Card** - Immutable data encapsulation
2. **Deck** - Collection encapsulation  
3. **GameEngine** - State and business logic encapsulation

Each layer protects its data and exposes only controlled interfaces. The GUI layer never touches private data directly.

### Key Concept
> **Encapsulation** = Bundling data with methods that operate on that data, while restricting direct access to internal state.

---

## 📌 Understanding Type Hints (Quick Reference)

Before we dive into encapsulation, you'll notice syntax like `-> None` and `: str` throughout the code. These are **type hints** (type annotations) - they document what types functions expect and return.

### Common Type Hints You'll See:

```python
def start_new_game(self) -> None:
    # -> None means "this function returns nothing"
    pass

def __init__(self, rank: str, suit: str) -> None:
    # rank: str means "rank parameter should be a string"
    # suit: str means "suit parameter should be a string"
    # -> None means "__init__ doesn't return anything"
    pass

def draw(self) -> Card | None:
    # -> Card | None means "returns either a Card or None"
    pass

self.__cards: list[Card] = []
    # : list[Card] means "this variable is a list of Card objects"
```

### Why Use Type Hints?

1. **Documentation**: Makes code easier to understand
2. **IDE Support**: Better autocomplete and error detection
3. **Clarity**: Shows what the function expects and returns

### Important Note:
Type hints are **optional** in Python - your code will run without them. They're documentation that helps developers (including you!) understand the code better.

**For this course:** Focus on understanding encapsulation concepts. Type hints are a bonus that makes professional code more readable!

---

### Interactive Type Hints Demo

In [ ]:
# Example 1: Function that returns nothing (-> None)
def greet(name: str) -> None:
    """This function prints a greeting but returns nothing."""
    print(f"Hello, {name}!")
    # No return statement = returns None

result = greet("Alice")
print(f"Function returned: {result}")  # Prints: None

print("\n" + "="*50 + "\n")

# Example 2: Function that returns a value
def add(a: int, b: int) -> int:
    """This function returns an integer."""
    return a + b

result = add(5, 3)
print(f"add(5, 3) returned: {result}")  # Prints: 8

print("\n" + "="*50 + "\n")

# Example 3: Function that might return None
def find_card(cards: list[str], target: str) -> str | None:
    """Returns the card if found, or None if not found."""
    for card in cards:
        if card == target:
            return card
    return None  # Explicitly return None when not found

my_cards = ["K♥", "Q♠", "A♦"]
found = find_card(my_cards, "Q♠")
print(f"Found: {found}")  # Prints: Q♠

not_found = find_card(my_cards, "2♣")
print(f"Not found: {not_found}")  # Prints: None

print("\n" + "="*50 + "\n")

# Example 4: Type hints don't enforce types!
def multiply(a: int, b: int) -> int:
    """Type hints say integers, but Python won't stop you from using strings!"""
    return a * b

# This works even though we said 'int':
result = multiply("Hello", 3)  # Type hint says int, but we passed string!
print(f"multiply('Hello', 3) = {result}")  # Prints: HelloHelloHello
print("⚠️ Python doesn't enforce type hints at runtime - they're just documentation!")

## 1. Card Class: Immutable Data Encapsulation

The `Card` class demonstrates:
- Private attributes with **name mangling** (`__attribute`)
- Read-only properties using `@property` decorator
- Immutability for data integrity

In [ ]:
class Card:
    """Immutable playing card with encapsulated rank and suit.

    Ranks are ordered for comparison: 2 < 3 < ... < 10 < J < Q < K < A
    """
    _RANKS = ["2","3","4","5","6","7","8","9","10","J","Q","K","A"]
    _SUITS = ["♣", "♦", "♥", "♠"]

    def __init__(self, rank: str, suit: str) -> None:
        rank = str(rank).upper()
        if rank not in self._RANKS:
            raise ValueError(f"Invalid rank: {rank}")
        if suit not in self._SUITS:
            raise ValueError(f"Invalid suit: {suit}")

        # Private (name-mangled) attributes
        self.__rank_index = self._RANKS.index(rank)
        self.__suit = suit

    # Read-only properties
    @property
    def rank(self) -> str:
        return self._RANKS[self.__rank_index]

    @property
    def suit(self) -> str:
        return self.__suit

    @property
    def value(self) -> int:
        """Numeric value for comparisons (read-only)."""
        return self.__rank_index

    # Comparison helpers
    def __lt__(self, other: "Card") -> bool:
        return self.value < other.value

    def __gt__(self, other: "Card") -> bool:
        return self.value > other.value

    def __eq__(self, other: object) -> bool:
        if not isinstance(other, Card):
            return NotImplemented
        return (self.value == other.value) and (self.suit == other.suit)

    def __str__(self) -> str:
        return f"{self.rank}{self.suit}"

    def __repr__(self) -> str:
        return f"Card(rank='{self.rank}', suit='{self.suit}')"

print("Card class defined successfully!")

### Demonstration: Card Encapsulation

In [ ]:
# Create some cards
king_hearts = Card("K", "♥")
ace_spades = Card("A", "♠")
two_clubs = Card("2", "♣")

# Access through properties (WORKS)
print(f"Card: {king_hearts}")
print(f"Rank: {king_hearts.rank}")
print(f"Suit: {king_hearts.suit}")
print(f"Value: {king_hearts.value}")

# Compare cards
print(f"\n{ace_spades} > {king_hearts}: {ace_spades > king_hearts}")
print(f"{two_clubs} < {king_hearts}: {two_clubs < king_hearts}")

### 🔒 Testing Data Protection

Let's see what happens when we try to break encapsulation:

In [ ]:
# Try to modify the card (SHOULD FAIL)
try:
    king_hearts.rank = "2"  # Try to change rank
except AttributeError as e:
    print(f"✓ Protection works! Error: {e}")

# Try to access private attribute directly (SHOULD FAIL)
try:
    print(king_hearts.__rank_index)  # Try to access private attribute
except AttributeError as e:
    print(f"✓ Name mangling works! Error: {e}")

# The sneaky way (you shouldn't do this!)
print(f"\n⚠️ Name-mangled attribute can still be accessed as:")
print(f"king_hearts._Card__rank_index = {king_hearts._Card__rank_index}")
print("But you should NEVER do this in real code!")

### 📝 Understanding the @property Decorator

The `@property` decorator transforms a method into a read-only attribute:

```python
@property
def rank(self) -> str:
    return self._RANKS[self.__rank_index]
```

**Benefits:**
1. **Clean syntax**: `card.rank` instead of `card.get_rank()`
2. **Read-only**: Cannot set `card.rank = "A"`
3. **Computed values**: Can add logic before returning
4. **Future flexibility**: Internal representation can change

---

## 2. Deck Class: Collection Encapsulation

The `Deck` class demonstrates:
- Private collection (`__cards`)
- Controlled access through public methods
- Preventing external manipulation of internal data

In [ ]:
import random

class Deck:
    """Encapsulated deck of 52 cards.

    The underlying list of cards is private. External code cannot mutate it.
    """
    def __init__(self) -> None:
        self.__cards: list[Card] = [
            Card(rank, suit) for suit in Card._SUITS for rank in Card._RANKS
        ]
        self.shuffle()

    def shuffle(self) -> None:
        """Shuffle the deck."""
        random.shuffle(self.__cards)

    def draw(self) -> Card | None:
        """Draw one card from the top, or None if empty."""
        if not self.__cards:
            return None
        return self.__cards.pop()

    @property
    def count(self) -> int:
        """Number of cards remaining."""
        return len(self.__cards)

print("Deck class defined successfully!")

### Demonstration: Deck Encapsulation

In [ ]:
# Create and use a deck
deck = Deck()
print(f"New deck has {deck.count} cards")

# Draw some cards
print("\nDrawing 5 cards:")
for i in range(5):
    card = deck.draw()
    print(f"  {i+1}. {card}")

print(f"\nCards remaining: {deck.count}")

# Shuffle
deck.shuffle()
print("\nDeck shuffled!")
print(f"Still {deck.count} cards remaining")

### 🔒 Testing Deck Protection

In [ ]:
# Try to access the cards list directly (SHOULD FAIL)
try:
    deck.__cards.clear()  # Try to empty the deck illegally
except AttributeError as e:
    print(f"✓ Protection works! Error: {e}")

# Try to modify count (SHOULD FAIL)
try:
    deck.count = 0  # Try to set count to zero
except AttributeError as e:
    print(f"✓ Read-only property works! Error: {e}")

print("\n💡 Key Point: External code can only use the public interface:")
print("   - deck.shuffle()")
print("   - deck.draw()")
print("   - deck.count (read-only)")
print("\nDirect manipulation of internal cards list is prevented!")

### 🤔 Why This Encapsulation Matters

**Without encapsulation (BAD):**
```python
# If __cards were public, someone could cheat:
deck.cards.clear()           # Empty the deck illegally
deck.cards.append(ace)       # Add a specific card (cheating!)
deck.cards = deck.cards[:10] # Only keep first 10 cards
```

**With encapsulation (GOOD):**
```python
# Only legal operations allowed:
deck.shuffle()      # OK - using public interface
card = deck.draw()  # OK - controlled card removal
print(deck.count)   # OK - read-only information
```

**Real-world parallel:** Think of a physical card dealer. You can't reach into the deck and rearrange cards (that's cheating!). You can only:
- Ask the dealer to shuffle
- Request one card at a time
- Ask how many cards are left

The `Deck` class enforces these same rules in code.

---

## 3. GameEngine Class: Business Logic Encapsulation

This is the **most sophisticated** encapsulation - it protects game state and enforces game rules.

The `GameEngine` demonstrates:
- Private state (deck, current card, score, game status)
- Read-only properties for state inspection
- Public methods for controlled state modification
- Internal helper methods (single underscore convention)

In [ ]:
class GameEngine:
    """Game rules and state for High-Low.

    Public API:
      - start_new_game()
      - guess_higher() / guess_lower() -> dict result
      - Read-only properties: current_card, score, deck_count, is_over
    """
    def __init__(self) -> None:
        self.__deck: Deck | None = None
        self.__current_card: Card | None = None
        self.__score: int = 0
        self.__over: bool = True

    def start_new_game(self) -> None:
        """Start a new game with a fresh, shuffled deck."""
        self.__deck = Deck()
        self.__score = 0
        self.__over = False
        self.__current_card = self.__deck.draw()
        if self.__current_card is None:
            self.__over = True

    def _ensure_playable(self) -> None:
        """Internal helper: Validate game state."""
        if self.__over or self.__deck is None:
            raise RuntimeError("Game is over or not started.")

    @property
    def current_card(self) -> Card | None:
        return self.__current_card

    @property
    def score(self) -> int:
        return self.__score

    @property
    def deck_count(self) -> int:
        return 0 if self.__deck is None else self.__deck.count

    @property
    def is_over(self) -> bool:
        return self.__over

    def _draw_next(self) -> Card | None:
        """Internal helper: Draw next card from deck."""
        if self.__deck is None:
            return None
        return self.__deck.draw()

    def _evaluate_guess(self, guess: str) -> dict:
        """Core game logic. Returns result dict for external use."""
        self._ensure_playable()
        next_card = self._draw_next()

        if next_card is None:
            self.__over = True
            return {
                'next_card': None,
                'outcome': 'deck_empty',
                'score': self.__score,
                'remaining': 0,
            }

        # Compare cards
        if next_card > self.__current_card:
            relation = 'higher'
        elif next_card < self.__current_card:
            relation = 'lower'
        else:
            relation = 'tie'

        if relation == 'tie':
            self.__current_card = next_card
            return {
                'next_card': next_card,
                'outcome': 'tie',
                'score': self.__score,
                'remaining': self.deck_count,
            }

        is_correct = (guess == relation)
        if is_correct:
            self.__score += 1
            self.__current_card = next_card
            return {
                'next_card': next_card,
                'outcome': 'correct',
                'score': self.__score,
                'remaining': self.deck_count,
            }
        else:
            self.__over = True
            return {
                'next_card': next_card,
                'outcome': 'wrong',
                'score': self.__score,
                'remaining': self.deck_count,
            }

    def guess_higher(self) -> dict:
        """Public method: Guess next card is higher."""
        return self._evaluate_guess('higher')

    def guess_lower(self) -> dict:
        """Public method: Guess next card is lower."""
        return self._evaluate_guess('lower')

print("GameEngine class defined successfully!")

### Demonstration: GameEngine Encapsulation

In [ ]:
# Create and start a game
game = GameEngine()
game.start_new_game()

print("=== High-Low Card Game ===")
print(f"Current card: {game.current_card}")
print(f"Score: {game.score}")
print(f"Cards remaining: {game.deck_count}")
print(f"Game over? {game.is_over}")

# Play a few rounds
print("\n=== Playing 3 rounds ===")
for i in range(3):
    if game.is_over:
        print("Game ended!")
        break

    current = game.current_card
    # Random guess for demo
    result = game.guess_higher() if i % 2 == 0 else game.guess_lower()

    print(f"\nRound {i+1}:")
    print(f"  Was: {current}")
    print(f"  Next: {result['next_card']}")
    print(f"  Outcome: {result['outcome']}")
    print(f"  Score: {result['score']}")

print(f"\nFinal score: {game.score}")

### 🔒 Testing GameEngine Protection

In [ ]:
# Try to cheat by modifying score (SHOULD FAIL)
try:
    game.score = 1000
except AttributeError as e:
    print(f"✓ Can't cheat with score! Error: {e}")

# Try to access private state (SHOULD FAIL)
try:
    game.__score = 999
except AttributeError as e:
    print(f"✓ Private state protected! Error: {e}")

# Try to call private method from outside (convention violation)
print("\n⚠️ Single underscore methods (_method) are conventionally private:")
print("   You CAN call them, but you SHOULDN'T:")
try:
    game._ensure_playable()  # This works but violates convention
    print("   game._ensure_playable() technically works...")
    print("   But it's against Python conventions to call it!")
except RuntimeError as e:
    print(f"   But may raise error if state invalid: {e}")

### 💡 Why GameEngine Encapsulation Matters

**Enforces game rules at the code level:**

```python
# BAD (if state were public):
engine.score = 1000          # Cheat! Set high score
engine.over = False          # Continue finished game
engine.current_card = ace    # Rig the game
```

```python
# GOOD (with encapsulation):
result = engine.guess_higher()  # Only legal way to play
# All rule validation happens inside _evaluate_guess()
# Score only increases through correct guesses
# Game only ends when rules dictate
```

**Prevents invalid state:**
- Score can't go up without a correct guess
- Game can't continue after it should end
- Current card can't change without drawing
- All state transitions validated by game logic

---

## 🔍 Encapsulation Patterns Comparison

| Class | Private Data | Public Interface | Why Encapsulate? |
|-------|--------------|------------------|------------------|
| **Card** | `__rank_index`, `__suit` | `rank`, `suit`, `value` properties | Immutability - cards shouldn't change |
| **Deck** | `__cards` list | `shuffle()`, `draw()`, `count` property | Prevent cheating - control card access |
| **GameEngine** | `__deck`, `__current_card`, `__score`, `__over` | `guess_higher()`, `guess_lower()`, properties | Game integrity - enforce rules |

### Common Pattern

All three classes follow this pattern:
1. **Hide data** with private attributes (`__attribute`)
2. **Expose information** through read-only properties (`@property`)
3. **Control behavior** through public methods
4. **Validate internally** before modifying state

---

## 📚 Deep Dive: The @property Decorator

### What @property Actually Does

The `@property` decorator is **syntactic sugar** for creating getters without explicit method calls.

**Without @property (verbose):**
```python
class Card:
    def get_rank(self):  # Explicit getter
        return self.__rank

card = Card("K", "♥")
print(card.get_rank())  # Need parentheses
```

**With @property (clean):**
```python
class Card:
    @property
    def rank(self):  # Property decorator
        return self.__rank

card = Card("K", "♥")
print(card.rank)  # No parentheses - looks like attribute!
```

### Properties Can Add Logic

Properties aren't just simple returns - they can compute values:

In [ ]:
class Temperature:
    """Demonstrates properties with validation logic."""

    def __init__(self, celsius):
        self.__celsius = celsius

    @property
    def celsius(self):
        """Get temperature in Celsius."""
        return self.__celsius

    @property
    def fahrenheit(self):
        """Computed property - converts to Fahrenheit."""
        return (self.__celsius * 9/5) + 32

    @property
    def kelvin(self):
        """Computed property - converts to Kelvin."""
        return self.__celsius + 273.15

# Demo
temp = Temperature(20)
print(f"Temperature:")
print(f"  {temp.celsius}°C")
print(f"  {temp.fahrenheit}°F")
print(f"  {temp.kelvin}K")

print("\n💡 All three properties read from one private attribute!")
print("   Encapsulation + computation = powerful abstraction")

### Properties with Setters (Controlled Write Access)

In [ ]:
class ValidatedTemperature:
    """Temperature with validated setter."""

    def __init__(self, celsius):
        self.celsius = celsius  # Uses setter validation

    @property
    def celsius(self):
        return self.__celsius

    @celsius.setter
    def celsius(self, value):
        """Setter with validation."""
        if value < -273.15:  # Absolute zero
            raise ValueError("Temperature below absolute zero!")
        self.__celsius = value

# Demo
temp = ValidatedTemperature(20)
print(f"Initial: {temp.celsius}°C")

temp.celsius = 30  # Uses setter
print(f"Updated: {temp.celsius}°C")

try:
    temp.celsius = -300  # Invalid!
except ValueError as e:
    print(f"✓ Validation works! Error: {e}")

### When to Use @property vs Regular Methods

**Use @property when:**
- Accessing data (read or computed)
- No parameters needed
- Fast operation (feels like attribute access)
- Logically represents object state

**Use regular methods when:**
- Performing actions or operations
- Parameters required
- Potentially slow operation
- Side effects or state changes

**Examples:**
```python
# Property - data access
score = game.score
count = deck.count

# Method - action
deck.shuffle()
result = game.guess_higher()
```

---

## 💭 Discussion Questions

Think about these questions as you review the code:

1. **Card Immutability:**
   - Why is it important that cards can't be modified after creation?
   - What could go wrong in a card game if cards were mutable?

2. **Deck Protection:**
   - Why shouldn't external code access `__cards` directly?
   - What's the difference between `deck.count` and `deck.draw()`?
   - Why is one a property and the other a method?

3. **GameEngine Design:**
   - Why are there so many private attributes?
   - What game rules are enforced by encapsulation?
   - Could a player cheat without encapsulation?

4. **Decorator Choice:**
   - When would you use `@property` vs a regular method?
   - Why doesn't this program use property setters?
   - How do properties improve code readability?

5. **Your Domain:**
   - What data in your project domain needs encapsulation?
   - What business rules could you enforce through private attributes?
   - Where would read-only properties be useful?

---

## 🎯 Practice Exercises

### Exercise 1: Add Score Statistics

Add properties to `GameEngine` that compute statistics:
- `average_score` (if you track multiple games)
- `win_rate` (correct guesses / total guesses)
- `longest_streak` (most consecutive correct guesses)

These should be **computed properties** that derive from existing private state.

In [ ]:
# Your code here


### Exercise 2: Create a Library Book Class

Design a `Book` class for a library management system with:
- Private attributes: `__isbn`, `__title`, `__author`, `__checked_out`
- Read-only properties: `isbn`, `title`, `author`, `is_available`
- Methods: `checkout()`, `return_book()`
- Validation: ISBN must be valid format, can't checkout if already checked out

In [ ]:
# Your code here


### Exercise 3: Break and Fix Encapsulation

1. Copy the `Card` class below
2. Make all attributes public (remove `__`)
3. Remove the `@property` decorators
4. Write code that demonstrates why this is problematic
5. Fix it by restoring proper encapsulation

In [ ]:
# Your code here


## 📝 Summary: Encapsulation Principles

### Three Key Techniques

1. **Private Attributes (`__attribute`)**
   - Name mangling prevents external access
   - Protects data integrity
   - Forces use of controlled interfaces

2. **Properties (`@property`)**
   - Clean attribute-like access
   - Read-only by default
   - Can add computation and validation

3. **Controlled Methods**
   - Public methods for operations
   - Private methods (`_method`) for internal logic
   - Validation before state changes

### The Encapsulation Pattern

```python
class EncapsulatedClass:
    def __init__(self, value):
        self.__private_data = value  # Hide data
    
    @property
    def data(self):  # Expose read-only
        return self.__private_data
    
    def modify_data(self, new_value):  # Controlled modification
        if self._validate(new_value):  # Internal validation
            self.__private_data = new_value
    
    def _validate(self, value):  # Private helper
        # validation logic
        return True
```

### Why Encapsulation Matters

- **Data Integrity**: Invalid states become impossible
- **Maintainability**: Internal changes don't break external code
- **Security**: Prevents unauthorized data manipulation
- **Clarity**: Clear public interface, hidden complexity

### Applying to Your Project

As you work on your team projects, ask:
1. What data should users never modify directly?
2. What business rules need code-level enforcement?
3. Where would read-only properties improve clarity?
4. What operations need validation before execution?

**Remember:** Good encapsulation makes invalid states impossible, not just difficult!

---

*End of Week 5 Encapsulation Notebook*